<h1  align="center"><b> MODELO ARMA </b></h1>

`Objetivo Geral:` Importar a série temporal transformada dos dados pluviométricos do município de São Paulo e realizar a modelagem ARMA (Auto-Regressiva de Média Móvel) para previsão.

`Dados:` Os dados foram transformados na pasta [Transformação e Decomposição](../[3]%20Transformação%20e%20Decomposição%20-%20Projeto%20Chuva/) do Projeto Chuva.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

<h2 align="center"><b> Importando Dados e Escolhendo o Modelo </b></h2>
&emsp;&emsp; O modelo ARMA é a combinação de um modelo AR (Auto-Regressivo) e um modelo MA (Média Móvel). Ou seja, o modelo ARMA é um modelo ARIMA sem a parte de diferenciação. Para escolher o melhor modelo utilizaremos o critério de informação de Akaike (AIC) que quanto menor melhor.

`Observação:` O modelo ARMA(p, q) é equivalente a ARIMA(p,0,q).

In [3]:
série_chuva = pd.read_csv('../[3] Transformação e Decomposição - Projeto Chuva/Série Transformada - Chuva Mensal.csv', sep = ';', index_col = 0)
série_chuva = pd.Series(série_chuva['Chuva Mensal (mm)'])
série_chuva.index = pd.date_range('1985', periods = len(série_chuva), freq = 'M')

In [4]:
from statsmodels.tsa.arima.model import ARIMA
modelo_ARMA = ARIMA(série_chuva, order = (2, 0, 2)) # Por inspeção o modelo ARMA(2, 0, 2) é um bom modelo.

resultado_ARMA = modelo_ARMA.fit() # Treinando o modelo
print(resultado_ARMA.summary()) # Sumário do modelo

                               SARIMAX Results                                
Dep. Variable:      Chuva Mensal (mm)   No. Observations:                  456
Model:                 ARIMA(2, 0, 2)   Log Likelihood                -654.518
Date:                Sun, 24 Dec 2023   AIC                           1321.036
Time:                        20:03:53   BIC                           1345.771
Sample:                    01-31-1985   HQIC                          1330.779
                         - 12-31-2022                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.7739      0.050     95.609      0.000       4.676       4.872
ar.L1          1.7316      0.000   4714.209      0.000       1.731       1.732
ar.L2         -1.0000      0.000  -9976.436      0.0

c:\Users\Atual\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<h2 align="center"><b> Análise de Resíduos </b></h2>

In [ ]:
resíduo = resultado_ARMA.resid
resíduo.plot() # Plotando o resíduo
plt.show()

![ARMA_Residuos](./Gráficos/ARMA_Residuos.png)

### `Normalidade dos Resíduos:`

In [13]:
import scipy.stats as stats
import seaborn as sns

In [ ]:
stats.probplot(resíduo, dist = 'norm', plot = plt)
plt.title('Normal QQ Plot - Resíduos')
plt.show()

sns.histplot(resíduo, kde = True)
plt.title('Histograma - Resíduos')
plt.show()

![Normal QQ Plot - Resíduos](./Gráficos/Normal%20QQ%20Plot%20-%20Resíduos.png)
![Histograma - Resíduos](./Gráficos/Histograma%20-%20Resíduos.png)

In [15]:
def teste_shapiro(série):
    e, p = stats.shapiro(série)
    print(f'Estatística de Teste = {e}')
    print(f'p-valor = {p}')
    print(f'Resultado: {"É Normal" if (p > 0.05) else "Não é Normal"}')

teste_shapiro(resíduo)

Estatística de Teste = 0.9914759993553162
p-valor = 0.010082758963108063
Resultado: Não é Normal


&emsp;&emsp; Esse modelo ficou melhor que o melhor modelo que tínhamos até então (AR(7), AIC = 1397.272, p-valor = 0.000947). A distribuição ainda não é normal, mas se aproxima bastante. 

### `Autocorrelação dos Resíduos:`

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

plot_acf(resíduo, lags = 20)
plt.title('Função de Autocorrelação - Resíduos')
plt.show()

plot_pacf(resíduo, lags = 20)
plt.title('Função de Autocorrelação Parcial - Resíduos')
plt.show()

![Função de Autocorrelação - Resíduos](./Gráficos/Função%20de%20Autocorrelação%20-%20Resíduos.png)
![Função de Autocorrelação Parcial - Resíduos](./Gráficos/Função%20de%20Autocorrelação%20Parcial%20-%20Resíduos.png)

&emsp;&emsp; As funções de autocorrelação mostram que os resíduos não possuem correlação com os valores passados, o que é um ótimo sinal.

<h2 align="center"><b> Previsão </b></h2>

In [5]:
tamanho_série = len(série_chuva) # Tamanho da série
previsão = resultado_ARMA.predict(start = tamanho_série, end = tamanho_série + 11) # Previsão para os próximos 12 meses

In [18]:
previsão2 = resultado_ARMA.forecast(steps = 12) # Método alternativo para previsão

In [ ]:
plt.plot(série_chuva, label = 'Série Original')
plt.plot(série_chuva - resíduo, label = 'Resíduo')
plt.plot(previsão, label = 'Previsão')
plt.title('Previsão - Modelo ARMA(2, 2)')
plt.legend(loc = 'best')
plt.show()

![Previsão - Modelo ARMA(2, 2)](./Gráficos/Previsão%20-%20Modelo%20ARMA(2,%202).png)

&emsp;&emsp; Finalmente chegamos na previsão, mas esses valores passaram por uma transformação por raiz cúbica, então precisamos elevar os valores ao cubo para obter a previsão real.

<h2 align="center"><b> Finalização </b></h2>

In [7]:
previsão_final = previsão ** 3
previsão_final.name = 'Previsão ARMA(2, 2)'
previsão_final.to_csv('Previsão ARMA - Chuva Mensal.csv', sep = ';', header = True) # Salvando a previsão em um arquivo csv
display(previsão_final)

2023-01-31    270.316792
2023-02-28    238.354983
2023-03-31    171.122442
2023-04-30    103.467142
2023-05-31     57.454094
2023-06-30     34.913901
2023-07-31     29.278366
2023-08-31     37.591454
2023-09-30     63.991270
2023-10-31    114.613221
2023-11-30    184.482710
2023-12-31    247.828460
Freq: M, Name: Previsão ARMA(2, 2), dtype: float64

In [24]:
Erro_Quadrático_Médio = (previsão_final['2023-01-31'] - 377.60)**2
Erro_Quadrático_Médio += (previsão_final['2023-02-28'] - 452.00)**2
Erro_Quadrático_Médio += (previsão_final['2023-03-31'] - 138.20)**2
Erro_Quadrático_Médio += (previsão_final['2023-04-30 '] - 165.70)**2
Erro_Quadrático_Médio += (previsão_final['2023-05-31'] - 43.30)**2
Erro_Quadrático_Médio += (previsão_final['2023-06-30'] - 85.20)**2
Erro_Quadrático_Médio += (previsão_final['2023-07-31'] - 15.00)**2
Erro_Quadrático_Médio = Erro_Quadrático_Médio / 7

print(f'Erro Quadrático Médio = {Erro_Quadrático_Médio:.2f}') # Erro Quadrático Médio

Erro Quadrático Médio = 9291.94
